In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

https://www.kaggle.com/willkoehrsen/introduction-to-manual-feature-engineering/

In [ ]:
!pip install modin[ray]
import modin.pandas as pd

In [ ]:
%config Completer.use_jedi = False

Dataset
* ``bureau``: information about client's previous loans with other financial institutions reported to Home Credit. Each previous loan has its own row.
* ``bureau_balance``: monthly information about the previous loans. Each month has its own row.

In [ ]:
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
bureau.head()

In [ ]:
bureau.shape

In [ ]:
bureau.isnull().sum()

In [ ]:
bureau.describe()

In [ ]:
bureau.info()

In [ ]:
pre_loan_count = bureau.groupby('SK_ID_CURR',as_index=False)['SK_ID_BUREAU'].count()
pre_loan_count = pre_loan_count.rename(columns={'SK_ID_BUREAU':'prev_loan_count'})
pre_loan_count

In [ ]:
app = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app = app.merge(pre_loan_count, on='SK_ID_CURR', how='left')
app.prev_loan_count = app.prev_loan_count.fillna(0)
app.head()

In [ ]:
def kernel_plot(var_name, df):
    print('correlation of {0} with target {1}'.format(var_name, app['TARGET'].corr(app.prev_loan_count)))
    repaid_median = df.loc[df['TARGET']==0, var_name].median()
    not_repaid = df.loc[df['TARGET']==1, var_name].median()
    print('median value for repaid ',repaid_median)
    print('median value for not repaid ',not_repaid)
    sns.kdeplot(df.loc[df['TARGET']==0, var_name],label='target=0')
    sns.kdeplot(df.loc[df['TARGET']==1, var_name], label='target=1')
    plt.legend()
    plt.show()

In [ ]:
kernel_plot('prev_loan_count',app)

In [ ]:
kernel_plot('EXT_SOURCE_3',app)

## Aggregating Numeric Columns
To account for the numeric information in the bureau dataframe, we can compute statistics for all the numeric columns. To do so, we groupby the client id, agg the grouped dataframe, and merge the result back into the training data. The agg function will only calculate the values for the numeric columns where the operation is considered valid. We will stick to using 'mean', 'max', 'min', 'sum' but any function can be passed in here. We can even write our own function and use it in an agg call.

In [ ]:
bureau.head(3)

In [ ]:
bureau_agg = bureau.drop('SK_ID_BUREAU',axis=1).groupby('SK_ID_CURR',as_index=False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
bureau_agg.head()

In [ ]:
# List of column names
columns = ['SK_ID_CURR']

# Iterate through the variables names
for var in bureau_agg.columns.levels[0]:
    # Skip the id name
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in bureau_agg.columns.levels[1][:-1]:
            # Make a new column name for the variable and stat
            columns.append('bureau_%s_%s' % (var, stat))

In [ ]:
columns = ['SK_ID_CURR']

for var in bureau_agg.columns.levels[0]:
    if var != 'SK_ID_CURR':
        for stat in bureau_agg.columns.levels[1][:-1]:
            columns.append('bureau_%s_%s' %(var,stat))

In [ ]:
columns

In [ ]:
bureau_agg.columns=columns
bureau_agg.head()

In [ ]:
app = app.merge(bureau_agg, on='SK_ID_CURR', how='left')
app.head()

In [ ]:
new_corr = []
for col in columns:
    corr = app['TARGET'].corr(app[col])
    new_corr.append([col,corr])

In [ ]:
new_corr = pd.DataFrame(new_corr).sort_values(1,ascending=False)

In [ ]:
new_corr.head(15)

In [ ]:
kernel_plot('bureau_DAYS_CREDIT_mean',app)

In [ ]:
app.describe()

In [ ]:
app.dtypes

## Function for Numeric Aggregations

In [ ]:
def agg_numeric(df, group_var, df_name):
    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [ ]:
def agg_numeric(df,group_var,df_name):
    # remove id variable other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
    
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # group by group_var and create their aggergates
    agg = numeric_df.groupby(group_var).agg(['count','mean','max','min','sum']).reset_index()

    #create new columns
    columns = [group_var]
    
    # iter through variable names
    for var in agg.columns.levels[0]:
        if var != group_var:
            for stat in agg.columns.levels[1][:-1]:
                columns.append('%s_%s_%s'%(df_name, var, stat))
    agg.columns = columns
    return agg
    
    """
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """

In [ ]:
bureau_agg_new = agg_numeric(bureau.drop('SK_ID_BUREAU',axis=1),group_var='SK_ID_CURR',df_name='bureau')
bureau_agg_new.head()

In [ ]:
bureau_agg_new.bureau_DAYS_CREDIT_count.dtype == 'int64'

In [ ]:
def target_corr(df):
    corr = []
    for col in df:
        if df[col].dtype=='int64':
            if col != 'TARGET':
                corr_val = df['TARGET'].corr(df[col])
                corr.append([col,corr_val])
    corr_df = pd.DataFrame(corr)
#     ,columns={0:'Coumns name',1:'correlation_with_target'}
    corr_df = corr_df.sort_values(1,ascending=False)
    return corr_df

In [ ]:
target_corr(app)

## Categorical Variables

In [ ]:
bureau.select_dtypes('object')

In [ ]:
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
categorical.head()

In [ ]:
categorical_group = categorical.groupby('SK_ID_CURR').agg(['sum','mean'])
categorical_group.head()

In [ ]:
categorical_group.columns.levels[0][:10]

In [ ]:
group_var = 'SK_ID_CURR'
columns = []
for var in categorical_group.columns.levels[0]:
    if var != group_var:
        for stat in ['sum','mean']:
            columns.append('%s_%s'%(var,stat))
categorical_group.columns = columns
categorical_group.head()

In [ ]:
app = app.merge(categorical_group, on='SK_ID_CURR', how='left')
app.head()

In [ ]:
app.shape

### categorical function

In [ ]:
def count_categorical(df,group_var,df_name):
    """
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    categorical = pd.get_dummies(df.select_dtypes('object'))
    categorical[group_var] = df[group_var]
    categorical_group = categorical.groupby(group_var).agg(['sum','mean'])
    
    columns = []
    for var in categorical_group.columns.levels[0]:
        if var != group_var:
            for stat in ['sum','mean']:
                columns.append('%s_%s_%s'%(df_name,var,stat))
    categorical_group.columns = columns
    return categorical_group

In [ ]:
bureau_counts = count_categorical(bureau, group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_counts.head()

In [ ]:
import pandas as pds

In [ ]:
description = pds.read_csv('../input/home-credit-default-risk/HomeCredit_columns_description.csv')
description

# Read bureau_balance

In [ ]:
bureau_balance = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')
bureau_balance.head()

In [ ]:
print(description.loc[description['Row']=='STATUS']['Description'])

In [ ]:
bureau_balance.STATUS.value_counts()

In [ ]:
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

In [ ]:
bureau_balance_agg = agg_numeric(bureau_balance,'SK_ID_BUREAU','bureau_balance')
bureau_balance_agg.head()

In [ ]:
bureau_balance_counts = count_categorical(bureau_balance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
bureau_balance_counts.head()

In [ ]:
bureau_balance_agg = agg_numeric(bureau_balance,'SK_ID_BUREAU','bureau_balance')
bureau_balance_agg.head()

In [ ]:
# Dataframe grouped by the loan
bureau_by_loan = bureau_balance_agg.merge(bureau_balance_counts, right_index=True, left_on='SK_ID_BUREAU', how='outer')
# Merge to include the SK_ID_CURR
bureau_by_loan = bureau[['SK_ID_BUREAU','SK_ID_CURR']].merge(bureau_by_loan, on='SK_ID_BUREAU', how='left')
# Aggregate the stats for each client
bureau_balance_by_client = agg_numeric(bureau_by_loan.drop(columns=['SK_ID_BUREAU']), group_var='SK_ID_CURR',df_name='client')

## Insert Computed Features into Training Data

In [ ]:
original_features = list(app.columns)
print('Total number of original features are',len(original_features))

WE already merge some columns so to avoid any complication i fist read app_train as a new df and then merge new calculated featues into it

In [ ]:
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
app_train.head(3)

In [ ]:
original_features = list(app_train.columns)
print('Total number of original features are',len(original_features))

In [ ]:
app_train = app_train.merge(bureau_counts, on='SK_ID_CURR',how='left')
app_train = app_train.merge(bureau_agg, on='SK_ID_CURR', how='left')
app_train = app_train.merge(bureau_balance_by_client, on='SK_ID_CURR',how='left')

In [ ]:
new_features = list(app_train.columns)
print('Number of features using previous loans from other institutions data: ', len(new_features))

## Feature Engineering Outcomes

In [ ]:
def missing_values_table(df):
    miss_val = df.isnull().sum()
    miss_val_table = pd.DataFrame(miss_val)
#     miss_val_table = miss_val_table.rename(columns={0:'Name of columns',1:'per_of_null'})
    miss_val_table['per_of_null'] = 100*df.isnull().sum()/len(df)
    miss_val_table = miss_val_table.sort_values('per_of_null',ascending=False)
    print('Dataframe has '+ str(len(df.columns))+ ' columns and Null columns are ' + str(len(miss_val_table[miss_val_table['per_of_null']!=0])))
    return miss_val_table

In [ ]:
miss = missing_values_table(app_train)
miss.head()

In [ ]:
# take out columns name that has more than 90 data null
miss_train_var = miss.index[miss['per_of_null']>90]
len(miss_train_var)

In [ ]:
app_train.to_csv('bureau+bureau_balance+app_train.csv',index=False)

# Correlations

First let's look at the correlations of the variables with the target. We can see in any of the variables we created have a greater correlation than those already present in the training data (from application).

In [ ]:
corr = app_train.corr()
corr = corr.sort_values('TARGET',ascending=False)
corr = pd.DataFrame(corr)
corr['TARGET'].head()

# Collinear Variables
We can calculate not only the correlations of the variables with the target, but also the correlation of each variable with every other variable. This will allow us to see if there are highly collinear variables that should perhaps be removed from the data.

Let's look for any variables that have a greather than 0.8 correlation with other variables.

In [ ]:
threshold = 0.8
above_threshold_var = {}
for col in corr:
    above_threshold_var[col] = list(corr.index[corr[col] > threshold])

In [ ]:
len(above_threshold_var)

In [ ]:
!lscpu |grep 'Model name'
!lscpu | grep 'Core(s) per socket'
!lscpu | grep 'Thread(s) per core'
!lscpu | grep MHz
!lscpu | grep 'L3 cache'
!cat /proc/meminfo | grep 'MemAvailable'
!df -h / | awk '{print $4}'

In [ ]:
cols_to_remove = []
cols_seen = []
cols_to_remove_pair = []

for key, value in above_threshold_var.items():
    cols_seen.append(key)
    for x in value:
        if x == key:
            next
        else:
            if x not in cols_seen:
                cols_to_remove.append(x)
                cols_to_remove_pair.append(key)
                
cols_to_remove = list(set(cols_to_remove))
print('Number of columns to remove: ', len(cols_to_remove))

In [ ]:
app_train_removed = app_train.drop(columns=cols_to_remove)
app_train_removed.shape

In [ ]:
app_train.to_csv('train_bureau_corrs_removed.csv', index = False)

## Modeling
To actually test the performance of these new datasets, we will try using them for machine learning! Here we will use a function I developed in another notebook to compare the features (the raw version with the highly correlated variables removed). We can run this kind of like an experiment, and the control will be the performance of just the application data in this function when submitted to the competition. I've already recorded that performance, so we can list out our control and our two test conditions:

**For all datasets, use the model shown below (with the exact hyperparameters).**

* control: only the data in the application files.
* test one: the data in the application files with all of the data recorded from the bureau and bureau_balance files
* test two: the data in the application files with all of the data recorded from the bureau and bureau_balance files with highly correlated variables removed.

In [ ]:
import lightgbm as lgb

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import gc

import matplotlib.pyplot as plt

In [ ]:
def model(features, encoding='ohe',n_folds=3):
    ids = features['SK_ID_CURR']
    labels = features['TARGET']
    features = features.drop(columns =['SK_ID_CURR','TARGET'])
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        cat_indices= 'auto'
    elif encoding == 'le':
        label_encoder = LabelEncoder()
        cat_indices = []
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                cat_indices.append(i)
                
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")    
    print('Training data shape: ',features.shape)
    feature_names = list(features.columns)
#     X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.33, random_state=42)
    features = np.array(features)
    k_fold = KFold(n_splits = n_folds, shuffle=False)

    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
                         
    for train_indices, valid_indices in k_fold.split(features):
        train_features, train_labels = features[train_indices], labels[train_indices]
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        model = lgb.LGBMClassifier(n_estimators=10000, objective='binary',
                                   class_weight='balanced', learning_rate=0.05,
                                   reg_alpha=0.1, reg_lambda=0.1,
                                   subsample=0.8, n_jobs=-1, random_state=50)
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        best_iteration = model.best_iteration_
#         feature_importance_values += model.feature_importance() / k_fold.n_splits
#         out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration=best_iteration)[:,1]
#         # Record the best score
#         valid_score = model.best_score_['valid']['auc']
#         train_score = model.best_score_['train']['auc']
        
#         valid_scores.append(valid_score)
#         train_scores.append(train_score)
        
#         # Clean up memory
#         gc.enable()
#         del model, train_features, valid_features
#         gc.collect()
#     # Make the feature importance dataframe
#     feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
#     # Overall validation score
#     valid_auc = roc_auc_score(labels, out_of_fold)
    
#     # Add the overall scores to the metrics
#     valid_scores.append(valid_auc)
#     train_scores.append(np.mean(train_scores))
    
#     # Needed for creating dataframe of validation scores
#     fold_names = list(range(n_folds))
#     fold_names.append('overall')
    
#     # Dataframe of validation scores
#     metrics = pd.DataFrame({'fold': fold_names,
#                             'train': train_scores,
#                             'valid': valid_scores}) 
    
#     return feature_importances, metrics

## Control
The first step in any experiment is establishing a control. For this we will use the function defined above (that implements a Gradient Boosting Machine model) and the single main data source (application).

In [ ]:
# train_control = pds.read_csv('../input/home-credit-default-risk/application_train.csv')

In [ ]:
fi, metrics = model(train_control)